In [ ]:
!pip install mira

This notebook uses `mira` to train a RetinaNet model to detect cats and dogs in images. Each cell is intended to be run in a separate kernel session. The first cell is most quickly executed on Google Colaboratory.

In [ ]:
from mira import datasets, detectors
from imgaug import augmenters as iaa
from keras import callbacks

# Load the Oxford pets datasets with a class
# for each breed.
dataset = datasets.load_oxfordiiitpets(breed=False)

# Load RetinaNet with pretrained backbone with MobileNet
detector = detectors.RetinaNet(
    input_shape=(224, 224, 3),
    pretrained_top=False,
    pretrained_backbone=True,
    annotation_config=dataset.annotation_config,
    backbone_name='mobilenet224_1_0'
)

# Split our dataset into training, validation,
# and test.
trainval, testing = dataset.train_test_split(
    train_size=0.7, test_size=0.3
)
training, validation = trainval.train_test_split(
    train_size=0.66, test_size=0.33
)

# Create an augmenter
augmenter = iaa.Sequential([
    iaa.Fliplr(0.5), # horizontally flip 50% of the images
    iaa.GaussianBlur(sigma=(0, 3.0)) # blur images with a sigma of 0 to 3.0
])

# Make a callback to stop the training job
# early if we plateau on the validation set.
cbs = [
    callbacks.EarlyStopping(
        monitor='val_loss',
        min_delta=0.1,
        patience=50,
        verbose=1,
        mode='auto',
        restore_best_weights=True
    )
]

# Run training job
detector.train(
    training=training,
    validation=validation,
    steps_per_epoch=50,
    epochs=1000,
    batch_size=8,
    augmenter=augmenter,
    callbacks=cbs
)
detector.model.save_weights('mobilenet224_1_0_oxfordcatdog.h5')

In [ ]:
import keras
from keras_retinanet import models

model = models.backbone(backbone_name='mobilenet224_1_0').retinanet(
    num_classes=2, inputs=keras.layers.Input((224, 224, 3))
)
model.load_weights('mobilenet224_1_0_oxfordcatdog.h5')
model.save('mobilenet224_1_0_oxfordcatdog_full.h5')

!tensorflowjs_converter \
    --input_format=keras \
    --output_format=tfjs_layers_model \
     mobilenet224_1_0_oxfordcatdog_full.h5 \
     mobilenet224_1_0_oxfordcatdog

In [ ]:
# Benchmarking
import keras
import numpy as np

from keras_retinanet import models

model = models.backbone(backbone_name='mobilenet224_1_0').retinanet(
    num_classes=2, inputs=keras.layers.Input((224, 224, 3))
)
model.load_weights('mobilenet224_1_0_oxfordcatdog.h5')
model = models.convert_model(
    model=model,
    nms=True,
    class_specific_filter=False
)
X = np.random.randint(low=0, high=255, size=(1, 224, 224, 3))
%timeit model.predict(X)